# Ádi_lentner megoldás inicializálás

In [1]:
import pandas as pd
import pickle
import requests

In [3]:
region_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-region_df.pkl"
    ).content
)
competition_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-competition_df.pkl"
    ).content
)
season_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-season_df.pkl"
    ).content
)
round_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-round_df.pkl"
    ).content
)
match_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-match_df.pkl"
    ).content
)
goal_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-goal_df.pkl"
    ).content
)
match_info_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-match_info_df.pkl"
    ).content
)
lineup_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-lineup_df.pkl"
    ).content
)
coach_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-coach_df.pkl"
    ).content
)
sidelined_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-sidelined_df.pkl"
    ).content
)
sub_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-sub_df.pkl"
    ).content
)
event_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-event_df.pkl"
    ).content
)
player_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-player_df.pkl"
    ).content
)

In [4]:
data = {
    "region": region_df,
    "competition": competition_df,
    "season": season_df,
    "round": round_df,
    "match": match_df,
    "goal": goal_df,
    "match_inf": match_info_df,
    "lineup": lineup_df,
    "coach": coach_df,
    "sidelined": sidelined_df,
    "sub": sub_df,
    "event": event_df,
    "player": player_df,
}

In [5]:
pickle.dump(data, open("data.pkl", "wb"))

In [2]:
data = pickle.load(open("data.pkl", "rb"))
match_info_all = (
    data["match"]
    .merge(data["round"], how="left")
    .merge(data["season"], how="left")
    .merge(data["competition"], how="left")
    .merge(data["region"], how="left")
)

# Első feladat:
A csapat megoldása - 

In [21]:
#goal_id néven indexeli sorban a goal_df eseményeit (nincs sok értelme, mert egyenlő az indexel)
data["goal"]["goal_id"] = [i for i in range(data["goal"].shape[0])]
#g_match néven új df a megfelelő oszlopokkal (balról)
g_match = pd.merge(
    data["goal"], match_info_all[["match_id", "region_id", "season_id"]], how="left"
)
# a goal_n df-ben azon scorer_id-k kerülnek, amik 20-nál több góljuk van
goal_n = g_match.groupby("scorer_id")["goal_id"].nunique().loc[lambda x: x >= 20]
#region_n df-be a 4-nél több régióban játszó játékosok
region_n = g_match.groupby("scorer_id")["region_id"].nunique().loc[lambda x: x >= 4]


#jupyter-kód
#a goal_n és region_n key-k közötti metszetet keresi
len(set(list(goal_n.keys())).intersection(list(region_n.keys())))

Endre megoldása -

In [ ]:
def _get_event_df(event_raw):

    return (
    #összeállít egy df-et a ...- okból
        pd.concat(
            [
                #az outputja egy olyan df, amelyben a match_id + player_id + time(minute) +\
                #event(goal, assist, y-card, 2nd-y-card, red-card), minden külön változót külön oszlopokba rendez
                #létrehoz egy külön oszlopot az első félidóben szerzett góloknak
                event_raw.pipe(lambda df: pd.concat([df, _fh_goals(df)])),
                pd.read_hdf("goal_df.h5")
                .dropna(subset=["assister_id"])
                .assign(
                    event="assist",
                    time=lambda df: df["minute"],
                    person_id=lambda df: df["assister_id"].astype(int),
                )
                .loc[:, ["match_id", "person_id", "time", "event"]],
            ]
        )
        #groupby helyettesítője -> match_id-n belül person_id-re szűr, majd az egyedi oszlopok celláit kitölti
        #az aggfunc = "count" helyettesíti a count_values-t, megszámolja a darabszámokat a cellákban
        .pivot_table(
            columns=["event"],
            index=["match_id", "person_id"],
            values="time",
            aggfunc="count",
        )
        #új oszlopokat ad hozzzá, hogy az első/második sárgalapot, piroslapot int-ként tárolja
        .assign(
            yc=lambda df: (df["y-card"] > 0).astype(int),
            yc2=lambda df: (df["2nd-y-card"] > 0).astype(int),
            rc=lambda df: (df["r-card"] > 0).astype(int),
        )
        #új oszlopokat ad hozzá, a bookings néven, a meccsen adott lapok számát tárolja, a sent_off a 2nd-y-card, vagy 
        #r-card, míg a y-all a sárgalapokat csak
        .assign(
            bookings=lambda df: df.loc[:, ["yc", "yc2", "rc"]].sum(axis=1),
            sent_off=lambda df: df.loc[:, ["rc", "yc2"]]
            .sum(axis=1)
            .astype(bool)
            .astype(int),
            yc_all=lambda df: df.loc[:, ["yc", "yc2"]].sum(axis=1),
        )
        #a csoportosítást megtartva, de sorokba rendezve újra indexeli a df-et
        .loc[:, event_cols]
        .reset_index()
    )

In [ ]:
self.player_x_match = (
    #a lup_df egy df, amiben részint bool változók(starter, sidelined) és számok(comes_on, full_bench, sub_time) vannak
    # illetve ez kiegészítve a player_de-el, ahol a nevek, (...személyes adatok...) + láb vannak
    #a lup_df-hez hozzámergeli az get_eventtel legyártott nagy df-et
    _lup_df.merge(self._get_event_df(_event_raw), how="left")
    .fillna(0)
    .assign(
        #sub_time oszlopban a 0-kat végtelenre cseréli
        sub_time=lambda df: df.loc[:, "sub_time"].replace(0, np.inf),
        #hozzáad egy oszlopot azon gólokkal, 
        #amiket egy becserélt játékos lőtt (where függvény kicsit olyan mint a maszkolás)
        sub_goals=lambda df: np.where(df["comes_on"], df["goal"], 0),
        #hozzáad egy oszlopot ami a gólt és gólpasszt is szerző játékosoknál 1
        g_a=lambda df: (df["goal"] > 0) & (df["assist"] > 0),
        #oszlop azokról, akiket becseréltek és lecseréltek
        come_n_go=lambda df: ((df["sent_off"] > 0) & df["comes_on"]),
        #a regio indexet hozzárendeli egy új oszlopban minden eseményhez
        region_id=_ids_for_gb["region_id"].values,
        #a szezon indexet hozzárendeli egy új oszlopban minden eseményhez
        season_id=_ids_for_gb["season_id"].values,
        #a bíró indexet hozzárendeli egy új oszlopban minden eseményhez
        referee_id=_ids_for_gb["referee_id"].values,
        #az esemény dátumát hozzárendeli egy új oszlopban minden eseményhez
        date=_ids_for_gb["start_time"].values,
    )
)

In [ ]:
#jupyter-kód
#a kiegészített lup_df + event_df + player_df-et groupby-olja a leírt sorrendben
dr.player_x_match.groupby(["person_id", "First name", "Last name", "region_id"])[
        "goal"
    ]
    #az utolsó oszlopban szereplő gólokat összeadja csoportonként (játékos szint)
    .sum()
    #a 20-nál nagyobbakat kiválasztja
    .loc[lambda s: s > 20]
    .reset_index()
    #megkeresi a metszetet azokkal, akik az új groupby+sum után több mint 3 régióval rendelkeznek
    .loc[
        lambda df: df["person_id"].isin(
            df.groupby("person_id")["region_id"]
            .count()
            .loc[lambda _s: _s >= 4]
            .index.tolist()
        ),
        :,
    ]

# Második feladat
A csapat megoldása -

In [114]:
#kikeresi a felfüggesztett játékosokat, akiket mergel a match_info df match_id és referee_id oszlopaival
suspended = data["sidelined"].loc[data["sidelined"]["reason"] == "Suspended"]
susp = pd.merge(suspended, data["match_inf"][["match_id", "referee_id"]], how="left")
#a df-ben groupby-olja a játékosok és bírók szerint az értékeket és megszámolja
grouped = susp.groupby(["player_id", "referee_id"]).count()


#jupyter-kód
#megkeresi azon eseteket, ahol a játékosnak két találkozása volt a bíróval és mivel
#a df felfüggesztésre van szűrve, így a visszakapott adat az olyan játékosok száma
#akiket felfüggesztettek és volt olyan két ugyanazon bíró által vezetett meccsük. Ezt player szinten megszámolja.
len(grouped[grouped["match_id"] == 2].index.unique(level="player_id"))

528

Endre megoldása -

In [ ]:
#jupyter-kód
#a fent kifejtett nagy df-et csoportosítja játékos és bíró szerint, majd a leküldött oszlop értékeit megszámolja és
#megkeresi azon értékeket, ahol a leküldött játékost több, mint egyszer ugyanazon bíró küldte le
#de ez szerintem hibás, mert van 25 játékos, akit 2-nél többször küldött le ugyanazon bíró
dr.player_x_match.groupby(["person_id", "referee_id"])["sent_off"].sum().loc[
    lambda s: s > 1
].index.get_level_values(
    #játékos szinten megkeresi az egyedi értékeket
    "person_id"
).nunique()

# Negyedik feladat
A csapat megoldása -

In [121]:
sub = data["sub"].rename(columns={"sub_id": "player_id"})
lineup = data["lineup"].rename(columns={"person_id": "player_id"})

#létrehozzák az onfield df-et a sub és lineup df-ekből
onfield = pd.concat([sub, lineup], sort=True)

#létrehoznak egy df-et két oszloppal a player_df-ből, amiben csak a ballábasok szerepelnek
left = data["player"][["Foot", "player_id"]].loc[data["player"]["Foot"] == "Left"]
#hozzámergelik az onfield_hez, player_id alapján összerndezve
merged = onfield.merge(left, how="left", on="player_id")

#Jupyter-kód
#csoportosítják meccs és csapat(oldal) szintjén az adatot és megszámolják az értékeket, és veszik a maximumukat
#a kapott maxok közül(player_id, subbed_id, time, Foot) kiválasztják a foot maxot.
merged.groupby(["match_id","side"]).count().max()["Foot"]

12

Endre megoldása -

In [ ]:
#Jupyter-kód
team_x_match = (
    #a visszakapott df-et csoportosítja meccs és csapat(oldal) szerint
    player_x_match.groupby(["match_id", "side"])
    #a függvény által visszaadott df felsorolt oszlopait mind összeadja, kivéve a sub_time-ot, ahol
    #megkeresi a minimum értéket
    .agg(
        {
            "is_lefty": "sum",
            "penalty-miss": "sum",
            "yc_all": "sum",
            "bookings": "sum",
            "comes_on": "sum",
            "sub_time": "min",
        }
    )
    #a df-hez hozzáadja a nagy player_x_match df azon sorait, ahol a játékos kezdett a meccsen(starter == True)
    #ezt újracsoportosítja meccs és oldal szerint szintén
    #a nationality oszlopban alapján csak az egyedi értékeket hagyja meg
    .join(
        dr.player_x_match.loc[lambda df: df["starter"], :]
        .groupby(["match_id", "side"])
        .agg({"Nationality": "nunique"})
    )
    #pluszba hozzácsatolja a teljes match_info df-et az eddigikhez és újraindexeli
    .join(dr.match_info)
    .reset_index()
    .assign(
        **{
            #létrehoz egy új oszlopot, ahol egy oszlopban lesznek a home_team és away_team kódjai
            "team_id": lambda df: np.where(
                df["side"] == "home", df["home_team"], df["away_team"]
            ),
            #opposition névvel létrehozza az előző oszlopnak az inverzét
            "opposition": lambda df: np.where(
                df["side"] == "home", df["away_team"], df["home_team"]
            ),
            #újabb oszlopot rendel hozá, ahol az True, ahol a csapat a meccs félidőben vesztésre állt, de megnyerte
            "ht_comeback": lambda df: (df["ht_gd"] < 0) & (df["ft_gd"] > 0),
            #létrehoz egy olyan oszlopot, ahol ha home_team == True a recordra, azaz otthoni a csapat,
            #és a full_time gólkülönbség pozitív, akkor a ft_gd-t, ha nem otthoni és a csapat(away) vesztett, azaz a 
            #full_time gólkülönbség negatív, akkor azt, azaz az oszlopban mindig az otthoni csapat GYŐZTES gólkülönbsége
            #szerepel , ha nem nyert az otthoni csapat, akkor szerintem NaN-t dob.
            "win": lambda df: np.where(
                df["side"] == "home", df["ft_gd"] > 0, df["ft_gd"] < 0
            ),
        }
    )
    #a kezdési időpontok szerint sorbarendezi a df-et
    .sort_values("start_time")
)

#a táblában rászűr az is_lefty oszlopra, ahol kiválasztja a maxot és mivel az adat meccsenként és oldalanként
#van csoportosítva illetve az is_lefty oszlopban szummázva vannak az értékek, így a maxot kiválasztva pont a keresett
#értéket kapjuk vissza
team_x_match["is_lefty"].max()

# Ötödik feladat
A csapat megoldása -

In [ ]:
#a korábban megismert suspended df-ben gyűjtik a felfüggesztett játékosokat
suspended = data["sidelined"].loc[data["sidelined"]["reason"] == "Suspended"]

#a felfüggesztett játékosok df-et és a sub_dfd-et mergelik a megadott key-kel
result = pd.merge(sub, suspended, how="inner", on=["match_id", "player_id"])

#Jupyter-kód
#Megnézik a kapott df hosszát, ami azonban nem jó megoldást ad a rossz szűrés miatt
result.shape[0]

Endre megoldása -

In [ ]:
#Jupyter-kód
#létrehozza a person_df-et, amiben személyek szerint csoportosít, és többek között a person_x_match függvényben
#kiszámolt come_n_go-t is oszlopként hozzáadja. A sorok szintjén szummázza az adatot.
person_df = dr.player_x_match.groupby(["person_id", "First name", "Last name"])[
    ["come_n_go", "sub_goals", "full_bench", "fh_goal", "g_a"]
].sum()
#a létrejött df-nek come_n_go oszlopát szummázza, amennyiben az nagyobb nullánál.
(person_df["come_n_go"] > 0).sum()

# Hatodik feladat
A csapat megoldása -

In [151]:
#top5_teams néven létrehoznak egy új df-et az event_df és a mergelt sub/lineup df adataival
#side = home vagy away
top5_teams = (
    data["event"]
    .rename(columns={"person_id": "player_id"})
    .merge(
        onfield[["match_id", "player_id", "side"]],
        how="inner",
        on=["match_id", "player_id"],
    )
)

#hozzáadják a meccs_id adatokat is a megfelelő sorokhoz, plusz team néven egy új oszlopot, ahol a hazai team kódot 
#tárolják
top5_teams = pd.concat(
    [
        top5_teams.loc[top5_teams["side"] == "home"]
        .merge(data["match"][["match_id", "home_team"]], how="inner", on=["match_id"])
        .rename(columns={"home_team": "team"}),
        top5_teams.loc[top5_teams["side"] == "away"]
        .merge(data["match"][["match_id", "away_team"]], how="inner", on=["match_id"])
        .rename(columns={"away_team": "team"}),
    ]
)

#külön lépésben hozzáadnak egy team_a oszlopot, ahol az away team kódokat tárolják
top5_teams = pd.concat(
    [
        top5_teams.loc[top5_teams["side"] == "home"]
        .merge(data["match"][["match_id", "away_team"]], how="inner", on=["match_id"])
        .rename(columns={"away_team": "team_a"}),
        top5_teams.loc[top5_teams["side"] == "away"]
        .merge(data["match"][["match_id", "home_team"]], how="inner", on=["match_id"])
        .rename(columns={"home_team": "team_a"}),
    ]
)

#A top5 adatbázisból egy "penalty-miss" szűréssel létrehozzák a missed df-et
missed = top5_teams.loc[top5_teams["event"] == "penalty-miss"]
#A missed-et csoportosítják meccsenként és megszámolják
grouped_pen = missed.groupby(["match_id"]).count()

#Jupyter-kód
#A csoportosított adatot player_id-ra szűrik és a 2-nél nagyobb számok darabszámát adják vissza
len(grouped_pen.loc[grouped_pen["player_id"] >= 2].index)

Endre megoldása -

In [ ]:
#Jupyter-kód
#A team_x_match df-et meccsek szerint csoportosítja és a penalty miss oszlopait megszámolja
#a pipe-al rászűr egy soron belül a 2-nél nagyobb értékekre
team_x_match.groupby("match_id")["penalty-miss"].sum().pipe(lambda s: (s >=2).sum())

# Top5 kérdések összehasonlítása

## Összehasonlításhoz szükséges függvények, adatok, stb behívása mind a két féltől

In [20]:
!pip install tables tqdm

In [21]:
from tqdm.notebook import tqdm
import pandas as pd

In [22]:
dfs = ["region_df",
       "competition_df",
       "season_df",
       "round_df",
       "match_df",
       "goal_df",
       "match_info_df",
       "lineup_df",
       "coach_df",
       "sidelined_df",
       "sub_df",
       "event_df",
       "player_df"]

In [4]:
import pandas as pd
import pickle
import requests

region_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-region_df.pkl"
    ).content
)
    
region_df.to_hdf("region_df.h5", "data")    

competition_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-competition_df.pkl"
    ).content
)
competition_df.to_hdf("competition_df.h5", "data")

season_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-season_df.pkl"
    ).content
)
season_df.to_hdf("season_df.h5","data")

round_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-round_df.pkl"
    ).content
)
round_df.to_hdf("round_df.h5","data")

match_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-match_df.pkl"
    ).content
)
match_df.to_hdf("match_df.h5","data")

goal_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-goal_df.pkl"
    ).content
)
goal_df.to_hdf("goal_df.h5","data")

match_info_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-match_info_df.pkl"
    ).content
)
match_info_df.to_hdf("match_info_df.h5","data")

lineup_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-lineup_df.pkl"
    ).content
)
lineup_df.to_hdf("lineup_df.h5","data")

coach_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-coach_df.pkl"
    ).content
)
coach_df.to_hdf("coach_df.h5","data")

sidelined_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-sidelined_df.pkl"
    ).content
)
sidelined_df.to_hdf("sidelined_df.h5","data")

sub_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-sub_df.pkl"
    ).content
)
sub_df.to_hdf("sub_df.h5","data")

event_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-event_df.pkl"
    ).content
)
event_df.to_hdf("event_df.h5","data")

player_df = pickle.loads(
    requests.get(
        f"https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/soccerway-player_df.pkl"
    ).content
)
player_df.to_hdf("player_df.h5","data")

data = {
    "region": region_df,
    "competition": competition_df,
    "season": season_df,
    "round": round_df,
    "match": match_df,
    "goal": goal_df,
    "match_inf": match_info_df,
    "lineup": lineup_df,
    "coach": coach_df,
    "sidelined": sidelined_df,
    "sub": sub_df,
    "event": event_df,
    "player": player_df,
}

pickle.dump(data, open("data.pkl", "wb"))

data = pickle.load(open("data.pkl", "rb"))
match_info_all = (
    data["match"]
    .merge(data["round"], how="left")
    .merge(data["season"], how="left")
    .merge(data["competition"], how="left")
    .merge(data["region"], how="left")
)



C:\Users\Anna\Anaconda3\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['round_id', 'round_name']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)
C:\Users\Anna\Anaconda3\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['minute', 'score', 'assister_id']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)
C:\Users\Anna\Anaconda3\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['Half-time', 'Full-time', 'Extra-time', 'Penalties', 'Venue', 'Kick-off', 'referee_

In [23]:
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd

from rw_fuction import DataRepo #átnevezve

In [24]:
%%time
dr = DataRepo()

Wall time: 1min 15s


In [25]:
import pandas as pd
import pickle

data = pickle.load(open("data.pkl", "rb"))
#Key df creation
match_info_all = (
    data["match"]
    .merge(data["round"], how="left")
    .merge(data["season"], how="left")
    .merge(data["competition"], how="left")
    .merge(data["region"], how="left")
)
#Renames
sub = data["sub"].rename(columns={"sub_id": "player_id"})
lineup = data["lineup"].rename(columns={"person_id": "player_id"})
#igazolt keret
onfield = pd.concat([sub, lineup], sort=True)

top5_teams = (
    data["event"]
    .rename(columns={"person_id": "player_id"})
    .merge(
        onfield[["match_id", "player_id", "side"]],
        how="inner",
        on=["match_id", "player_id"],
    )
)
top5_teams = pd.concat(
    [
        top5_teams.loc[top5_teams["side"] == "home"]
        .merge(data["match"][["match_id", "home_team"]], how="inner", on=["match_id"])
        .rename(columns={"home_team": "team"}),
        top5_teams.loc[top5_teams["side"] == "away"]
        .merge(data["match"][["match_id", "away_team"]], how="inner", on=["match_id"])
        .rename(columns={"away_team": "team"}),
    ]
)
top5_teams = pd.concat(
    [
        top5_teams.loc[top5_teams["side"] == "home"]
        .merge(data["match"][["match_id", "away_team"]], how="inner", on=["match_id"])
        .rename(columns={"away_team": "team_a"}),
        top5_teams.loc[top5_teams["side"] == "away"]
        .merge(data["match"][["match_id", "home_team"]], how="inner", on=["match_id"])
        .rename(columns={"home_team": "team_a"}),
    ]
)


def get_top_n(df, group, sort, n):
    out = (
        df.groupby(group)
        .count()
        .sort_values(by=[sort], ascending=False)
        .iloc[0:n]
        .reset_index()
    )
    out["player_id"] = [int(i) for i in out["player_id"]]
    return out


def top_player(df):
    top = pd.merge(
        df,
        data["player"][["First name", "Last name", "player_id"]],
        how="inner",
        on="player_id",
    )
    return top

In [26]:
#1.
data["goal"]["goal_id"] = [i for i in range(data["goal"].shape[0])]
g_match = pd.merge(
    data["goal"], match_info_all[["match_id", "region_id", "season_id"]], how="left"
)
goal_n = g_match.groupby("scorer_id")["goal_id"].nunique().loc[lambda x: x >= 20]
region_n = g_match.groupby("scorer_id")["region_id"].nunique().loc[lambda x: x >= 4]
# 2.
suspended = data["sidelined"].loc[data["sidelined"]["reason"] == "Suspended"]
susp = pd.merge(suspended, data["match_inf"][["match_id", "referee_id"]], how="left")
grouped = susp.groupby(["player_id", "referee_id"]).count()
# 3.

# 4.
left = data["player"][["Foot", "player_id"]].loc[data["player"]["Foot"] == "Left"]
merged = onfield.merge(left, how="left", on="player_id")
# 5.
result = pd.merge(sub, suspended, how="inner", on=["match_id", "player_id"])
# 6.
missed = top5_teams.loc[top5_teams["event"] == "penalty-miss"]
grouped_pen = missed.groupby(["match_id"]).count()

# Összehasonlítás

## top5 sérülés idők

### Ádi&Letner

In [ ]:
def get_top_n(df, group, sort, n):
    out = (
        df.groupby(group)
        .count()
        .sort_values(by=[sort], ascending=False)
        .iloc[0:n]
        .reset_index()
    )
    out["player_id"] = [int(i) for i in out["player_id"]]
    return out

In [27]:
sort_reason = data["sidelined"].groupby(["player_id", "reason"]).count().reset_index()
top_reason = get_top_n(sort_reason, "player_id", "reason", 5)

In [28]:
top_player(top_reason)[["First name", "Last name", "reason"]]

,First name,Last name,reason
0,Chris,Smalling,15
1,Phil,Jones,15
2,Jonny,Evans,15
3,Wayne Mark,Rooney,15
4,Rafael,Pereira da Silva,14


### Endre

In [29]:
def _get_top5_sideline(self, sideline_df):
    #top5 reason ami miatt ki volt állítva valaki, újdonság: agg
        self.top5_sideline = (
            sideline_df.groupby("player_id")
            .agg({"reason": "nunique"})
            .nlargest(5, "reason")
            .reset_index()
            .merge(
                pd.read_hdf("player_df.h5").loc[
                    :, ["player_id", "First name", "Last name", "Nationality"]
                ],
                how="left",
            )
        )

In [30]:
dr.top5_sideline

,player_id,reason,First name,Last name,Nationality
0,193,15,Wayne Mark,Rooney,England
1,20687,15,Jonny,Evans,Northern Ireland
2,79474,15,Chris,Smalling,England
3,97299,15,Phil,Jones,England
4,22357,14,Mario,Barwuah Balotelli,Italy


Különbségek:
- Nagy különbség persze az, hogy classokba van redezve Edre kódja
- Agg idex használata endrénél, ami könnyebben aggregál, mint a count_value
- Top-5-öt nlargest szerit szedi ki Edre, ez picit gyorsabb és rövidebb, mint Ádiéké
- Eredménynek jót ad Ádiék kódja

# Többi top5 player többi feladatra

## Edre által használt kódok a feladatmegoldáshoz:

In [ ]:
#ez konkrétam az első félidős top5 feladatnál határozza meg az időt
def _is_first_half(_df):
    return (
        _df["time"].str.replace("'", "").str.split("+").str[0].astype(int)
        <= 45
    )

#ez is még ezt viszit tovább, szűri az első félidős gólokra az event df-et 
def _fh_goals(_df):
    return (
        _df.loc[lambda df: (df["event"] == "goal") & (df["time"] != "'"), :]
        .loc[_is_first_half, :]
        .assign(event="fh_goal")
    )


In [ ]:
def _get_goals_after_rc(self, event_raw, lup_raw):
        #először az evet df-et leszűrjük a goal, piroslap kés sárgára, mergeljük a lup_raw-val (melyik oldal?)
        sided_rel_events = (
            event_raw.loc[
                lambda df: df["event"].isin(["goal", "r-card", "2nd-y-card"])
                & (df["time"] != "'")
            ]
            .merge(
                lup_raw.loc[:, ["match_id", "person_id", "side"]], how="left",
            )
            .assign(
                event=lambda df: df["event"].replace("2nd-y-card", "r-card")
            )
        )
        #megcsináljuk az időt
        time_lists = (
            sided_rel_events["time"].str.replace("'", "").str.split("+")
        )
        #megcsiálja az oldalakat szeritit ismét
        pr_arr = (
            sided_rel_events.assign(
                main_time=time_lists.str[0].astype(int),
                plus_time=np.where(
                    time_lists.str.len() > 1, time_lists.str[1], "0"
                ).astype(int),
            )
            .sort_values(
                ["match_id", "side", "main_time", "plus_time", "event"]
            )
            .loc[:, ["match_id", "side", "event"]]
            .values

In [ ]:
def _get_participant_df(self):
    ##akik az adott meccsen a pályán voltak, nekik a nevei és hogy milyen lábasok
        _sidelined_raw = pd.read_hdf("sidelined_df.h5")
        #itt megadja hogy min szűrve nézz meg a top5-öt
        self._get_top5_sideline(_sidelined_raw)
        #lineupdf megkapja az időt és a sub_df-et concattal
        #egy olyan df-et kapok vissza, ahol a linup_df oszlopaihoz (hogy melyikhez azt a lup_cols lövi be), hozzátesszük a sub_df-et (ezt egymás alá)
        #és ehhez mergeljük hozzá a player infókat
        return (
            pd.concat(
                [
                    pd.read_hdf("lineup_df.h5")
                    .assign(
                        starter=True,
                        comes_on=False,
                        full_bench=False,
                        sidelined=False,
                        sub_time=np.inf,
                    )
                    .loc[:, lup_cols],
                    pd.read_hdf("sub_df.h5")
                    .rename(columns={"sub_id": "person_id"})
                    .assign(
                        starter=False,
                        comes_on=lambda df: ~df["subbed_id"].isna(),
                        full_bench=lambda df: df["subbed_id"].isna(),
                        sidelined=False,
                        sub_time=_parse_time_int,
                    )
                    .loc[:, lup_cols],  # note: no need for who was subbed off
                    _sidelined_raw.rename(columns={"player_id": "person_id"})
                    .assign(
                        starter=False,
                        comes_on=False,
                        full_bench=False,
                        sidelined=True,
                        sub_time=np.inf,
                    )
                    .loc[:, lup_cols],
                ]
            )
            .drop_duplicates(
                subset=["person_id", "match_id", "side"], keep="first"
            )
            .merge(
                pd.read_hdf("player_df.h5")
                .rename(columns={"player_id": "person_id"})
                .assign(is_lefty=lambda df: (df["Foot"] == "Left").astype(int))
                .loc[:, player_cols],
                how="left",
            )  # here astype(int) saves a LOT of time
        )

In [ ]:
def _get_event_df(event_raw):

        return (
            pd.concat(
                [
                    #első félidős gólok, assistolók
                    event_raw.pipe(lambda df: pd.concat([df, _fh_goals(df)])),
                    pd.read_hdf("goal_df.h5")
                    .dropna(subset=["assister_id"])
                    .assign(
                        event="assist",
                        time=lambda df: df["minute"],
                        person_id=lambda df: df["assister_id"].astype(int),
                    )
                    .loc[:, ["match_id", "person_id", "time", "event"]],
                ]
            )
            .pivot_table(
                columns=["event"],
                index=["match_id", "person_id"],
                values="time",
                aggfunc="count",
            )
            .assign(
                yc=lambda df: (df["y-card"] > 0).astype(int),
                yc2=lambda df: (df["2nd-y-card"] > 0).astype(int),
                rc=lambda df: (df["r-card"] > 0).astype(int),
            )
            .assign(
                bookings=lambda df: df.loc[:, ["yc", "yc2", "rc"]].sum(axis=1),
                sent_off=lambda df: df.loc[:, ["rc", "yc2"]]
                .sum(axis=1)
                .astype(bool)
                .astype(int),
                yc_all=lambda df: df.loc[:, ["yc", "yc2"]].sum(axis=1),
            )
            .loc[:, event_cols]
            .reset_index()
        )


In [ ]:
def _load_player_x_match(self):
        _lup_df = self._get_participant_df()
        #pályán lévő játékosok
        _event_raw = pd.read_hdf("event_df.h5")
        #események
        self.coach_df = pd.read_hdf("coach_df.h5").assign(
            date=lambda df: self.match_info.reindex(df["match_id"])[
                "start_time"
            ].values
        )
        
        self._get_goals_after_rc(_event_raw, _lup_df)
        #kiállítás utáni események
        _ids_for_gb = self.match_info.reindex(_lup_df["match_id"]).loc[
            :, ["region_id", "season_id", "referee_id", "start_time"]
        ]
        #ez csak leszűri az adatbázist
        
        #itt jön a mágia. Miután megvan az összes playerxmatch recordú adatbázis
        #azután már csak a különböző változókra szűrűnk
        self.player_x_match = (
            _lup_df.merge(self._get_event_df(_event_raw), how="left")
            .fillna(0)
            .assign(
                sub_time=lambda df: df.loc[:, "sub_time"].replace(0, np.inf),
                #top5 subként lőtt gól
                sub_goals=lambda df: np.where(df["comes_on"], df["goal"], 0),
                #top5 goal és assist is
                g_a=lambda df: (df["goal"] > 0) & (df["assist"] > 0),
                #akik le lettek külve csereként
                come_n_go=lambda df: ((df["sent_off"] > 0) & df["comes_on"]),
                #hány régió value...
                region_id=_ids_for_gb["region_id"].values,
                season_id=_ids_for_gb["season_id"].values,
                referee_id=_ids_for_gb["referee_id"].values,
                date=_ids_for_gb["start_time"].values,
            )
        )

In [31]:
person_df = dr.player_x_match.groupby(["person_id", "First name", "Last name"])[
    ["come_n_go", "sub_goals", "full_bench", "fh_goal", "g_a"]
].sum()

Wall time: 18.7 s


In [14]:
for c in person_df.columns[1:]:
    display(HTML(person_df[c].nlargest(5).to_frame().to_html()))

,,,sub_goals
person_id,First name,Last name,
119,Lionel Andrés,Messi Cuccittini,34.0
13690,Kevin,Gameiro,34.0
103171,Álvaro Borja,Morata Martín,34.0
53937,Javier,Hernández Balcázar,31.0
1595,Karim,Benzema,30.0


,,,full_bench
person_id,First name,Last name,
1310,Jessy,Moulin,363.0
18585,Daniele,Padelli,316.0
4336,Marco,Storari,310.0
97650,Rafael Pires,Monteiro,302.0
17587,Germán Darío,Lux,300.0


,,,fh_goal
person_id,First name,Last name,
119,Lionel Andrés,Messi Cuccittini,250.0
382,Cristiano Ronaldo,dos Santos Aveiro,235.0
2290,Luis Alberto,Suárez Díaz,138.0
3051,Sergio Leonel,Agüero del Castillo,137.0
1595,Karim,Benzema,133.0


,,,g_a
person_id,First name,Last name,
119,Lionel Andrés,Messi Cuccittini,93.0
382,Cristiano Ronaldo,dos Santos Aveiro,65.0
2290,Luis Alberto,Suárez Díaz,48.0
102697,Neymar,da Silva Santos Junior,44.0
1595,Karim,Benzema,38.0


In [15]:
%%time
dr.player_x_match.groupby(["person_id", "First name", "Last name", "season_id"])[
    "assist"
].sum().reset_index().sort_values(
    "assist", ascending=False
).drop_duplicates("person_id", keep="first").head(5)

Wall time: 8.68 s


,person_id,First name,Last name,season_id,assist
222302,67424,Luciano,Narsingh,5963,22.0
165062,40709,Dušan,Tadić,5087,21.0
2596,119,Lionel Andrés,Messi Cuccittini,5334,21.0
425904,218727,Zinedine,Ferhat,14061,20.0
157572,38568,Graham,Dorrans,3452,20.0


# Ádi&Lentner - Sub goal

In [32]:
# 2.
sub["sub"] = 1
g_match_2 = pd.merge(data["goal"], match_info_all["match_id"], how="left")
subbed_goals = pd.merge(
    g_match_2.rename(columns={"scorer_id": "player_id"}),
    sub[["match_id", "player_id", "sub"]],
    how="inner",
    on=["player_id", "match_id"],
).drop_duplicates()
##csak playerid szerint megy a groupby
top_subbed_goals = get_top_n(subbed_goals, "player_id", "goal_id", 5)

In [33]:
top_player(top_subbed_goals)[["First name", "Last name", "goal_id"]]

,First name,Last name,goal_id
0,Kevin,Gameiro,39
1,Lionel Andrés,Messi Cuccittini,38
2,Álvaro Borja,Morata Martín,36
3,Karim,Benzema,33
4,Javier,Hernández Balcázar,31


# Ádi&Lentner full bench

In [34]:
# 3.
#Subid hívása - isnull - itt szűri le hogy végig ültek, playerID szerint szűr, top5
top_sat = get_top_n(sub[sub["time"].isnull()], "player_id", "sub", 5)

In [35]:
top_player(top_sat)[["First name", "Last name", "sub"]]

,First name,Last name,sub
0,Jessy,Moulin,363
1,Daniele,Padelli,316
2,Marco,Storari,310
3,Rafael Pires,Monteiro,302
4,Germán Darío,Lux,300


# Ádi&Lentner first half

In [36]:
# 4.
##kiszedi azt, amelyik gólnak nincsen perce és azt átírja 0-ra - itt sztem azt ki kell dobni
ind = data["goal"].loc[data["goal"]["minute"] == "'"].index[0]
data["goal"].at[ind, "minute"] = "0'"
#csinál egy first half változót és ugyanazt mint mi csak több sorban
data["goal"]["first half"] = [int(i.split("'")[0]) for i in data["goal"]["minute"]]
first_half = data["goal"].loc[data["goal"]["first half"] <= 45]
first_half = first_half.rename(columns={"scorer_id": "player_id"})
## itt a minute nem tudom miért van de amúgy groupby
top_first = get_top_n(first_half, "player_id", "minute", 5)

In [37]:
top_player(top_first)[["First name", "Last name", "goal_id"]]

,First name,Last name,goal_id
0,Cristiano Ronaldo,dos Santos Aveiro,286
1,Lionel Andrés,Messi Cuccittini,278
2,Sergio Leonel,Agüero del Castillo,155
3,Luis Alberto,Suárez Díaz,152
4,Edinson Roberto,Cavani Gómez,152


# Ádi&Lentner assist&goal

In [38]:
work = data["goal"][["match_id", "scorer_id", "assister_id"]]
work = work.rename(columns={"scorer_id": "player_id"})
top_sa = get_top_n(work, ["player_id", "match_id"], "assister_id", 5)

In [39]:
top_player(top_sa)[["First name", "Last name", "assister_id"]]

,First name,Last name,assister_id
0,Erling,Håland,8
1,Mehmet,Aldoğan,5
2,Nahikari,García Pérez,5
3,Luka,Jović,5
4,Sergio Leonel,Agüero del Castillo,5


# Ádi&Lentner top5 assister in season

In [40]:
# TOP5
# 1.
#kidobja azt az egy missing valuet nyom egy renamet, g_match az egy gól_df + key_df
ip = g_match.dropna(subset=["assister_id"]).rename(columns={"assister_id": "player_id"})
#jön a groupby függvény 
topassists = get_top_n(ip, ["player_id", "season_id"], "goal_id", 6,)

In [41]:
top_player(topassists).rename(columns={"minute": "assist count"})[
    ["First name", "Last name", "season_id", "assist count"]
]

,First name,Last name,season_id,assist count
0,Luciano,Narsingh,5963,22
1,Dušan,Tadić,5087,21
2,Lionel Andrés,Messi Cuccittini,5334,21
3,Lionel Andrés,Messi Cuccittini,12401,21
4,Zinedine,Ferhat,14061,20
5,Kevin,De Bruyne,9643,20


# Külöbségek
Máshogy épül fel a struktúra teljesen, most így kiragadva fölöslegesen bonyolultban is tűnhet Edre kódja, viszont miután a player x match df létrejön, ahol minden oszlop más benne van, ami ilyen recordú kérdést tartalmas, onnantól jóval gyorsabb minden. Összehasonlításban most közösen veszem a kérdéseket, mivel Endre kódját fölösleges leszedni egy kérdésre.

Függvény szerinti különbségek:
Endre kódjában az alábbi függvények szerepelnek, amit Ádiék nem használnak:
- isin: value szerinti szűrés, event DF leszűrésére alkalmazza a piros laposok miatt
- assign: több oszlophoz hozzátenni egy meglévő dataframehez, nagyon gyakran használt parancs
- where: van np. és simplaa formában is használva, új értéket ad dataframe cellákhoz (time meghatároznál haszált), 
- pipe: olyan függvények összekötése, amik series, dataframe vagy grouby obejtet adnak 
- pivot table: egy multiindex szintű pivot table készítése: record: Match x person
- HTML to_frame, to_html display: csupán az árbázolást segíti

Eredmények:
- sub_goal: nevek azonosak,értékek nem
- full_bench: nevek és értékek is azonosak
- first_half goal: eltérő eredméy
- score_assist: eltérő eredményel
- top5 assist/season: Messit kivéve eltérő eredmények

# Tizenharmadik feladat
A csapat megoldása -

In [ ]:
#Az feladatban megismert missed df-et csapatok mentén csoportosítjuk 

#Jupyter-kód
#a team(hazai) szerint csoportosított df sorait oszloponként megszámoljuk, csökkenő sorrendbe rakja 
#és az első 5-öt veszi
missed.groupby("team").count().sort_values(by=["side"], ascending=False).iloc[0:5]

Endre megoldása -

In [ ]:
#Jupyter-kód
#a team_x_match nagy df-jét csoportosítja csapatonként, a penalty-miss oszlop értékeit szummázza és veszi a top5-öt
team_x_match.groupby("team_id")["penalty-miss"].sum().nlargest(5)

# Tizennegyedik feladat
A csapat megoldása -

In [ ]:
#Jupyter-kód
#a missed df-ben csoportosítás után kikeresik a top5 missed soroknak a team_a oszlopértékeit
missed.groupby("team_a").count().sort_values(by=["side"], ascending=False).iloc[0:5]

Endre megoldása -

In [ ]:
#Jupyter-kód
#ugyanúgy oldja, meg mint az előzőt, csak itt a team_x_match fv-vel létrehozott opposition oszlop szerint csoportosít
team_x_match.groupby("opposition")["penalty-miss"].sum().nlargest(5)

# Tanulságok

Egyik egyértelmű felismerés, hogy a tervezési szakaszt még alaposabban és a bemutatott kulcsok, rekordok alapján kell megcsinálni, majd a kérdéseket eszerint csoportosítani

Endre megoldásra arra is rámutat, hogy megfelelő háttértudással (típusok skálázódásai , stb.) lehet nagy DataFrame-ket építeni, amik gyorsak maradnak

A kulcsok mentén lehet általánosítani a kérdéseket és ezen aggregálási szinteken írni általános függvényeket, amiket minden kérdés tud használni

Classba rendezve és instance-ket használva könnyebb importálni a függvényeket egy notebookba

Memória hatékonyabb egy nagy jól megépített DF, mint sok kérdésenként létrehozott specifikus adattábla

Assign, pipe: ezekkel a fügvényekkel nagyon le lehet rövidíteni a kapcsolásokat/szűréseket,

Konkrét kérdéseknél rossz értelmezésünk volt (top5 season assiter) ezeket ki kell javítani

A classok használatára érdemes rávenni magunkat, mivel nagynak tűnik a betanulási fix költség, de hasznos lehet később.

A csapaton belüli feladatmegosztásra jobban kell figyelni ilyen általánosított feladatoknál, hogy ne dolgozzunk párhuzamosan ugyanazon.

A record - columns - key táblázatot a kérdésekre több sziten meg lehet csinálni. Érdemes nem az utolsó lépésnél ezt elvégezi, hanem a folyamat szempontjából előbb, így növelve az aggregálható kérdések számát.